# Calculate distance to stores in London by car

In [34]:
import boto3
from dotenv import load_dotenv
import os
import pandas as pd

In [35]:
# Load environment variables from .env file
load_dotenv()

# Create a client for Amazon Location service
amazon_location_client = boto3.client(
    "location",
    aws_access_key_id = os.getenv("AWS_ACCESS_KEY"),
    aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name = os.getenv("AWS_REGION"),
)

In [36]:
response = amazon_location_client.calculate_route_matrix(
    CalculatorName = 'AWS-Esri-Demo',
    TravelMode = 'Car',
    DeparturePositions=[
        [-0.1417089267, 51.5787616672]
    ],
    DestinationPositions = [
        [-0.1254974319, 51.50852485],
        [-0.1342243716, 51.52619485],
        [-0.1231958457, 51.53036755],
        [-0.08229997944, 51.51856002],
        [-0.1726801947, 51.51721979],
        [-0.1259365099, 51.53253756],
        [-0.1425202953, 51.49654998],
        [-0.1117242765, 51.50333369]
    ],
    DistanceUnit = 'Kilometers'
)

In [37]:
response['RouteMatrix']

[[{'Distance': 9.757, 'DurationSeconds': 1497},
  {'Distance': 7.567, 'DurationSeconds': 1030},
  {'Distance': 6.823, 'DurationSeconds': 929},
  {'Distance': 9.059, 'DurationSeconds': 1422},
  {'Distance': 10.065, 'DurationSeconds': 1446},
  {'Distance': 7.212, 'DurationSeconds': 1016},
  {'Distance': 12.44, 'DurationSeconds': 1792},
  {'Distance': 11.098, 'DurationSeconds': 1641}]]

In [38]:
response['Summary']

{'DataSource': 'Esri',
 'DistanceUnit': 'Kilometers',
 'ErrorCount': 0,
 'RouteCount': 8}